In [1]:
# RDD创建
"""
第一种：读取一个外部数据集。
比如，从本地文件加载数据集，或者从HDFS文件系统、HBase、Cassandra、Amazon S3等外部数据源中加载数据集。
Spark可以支持文本文件、SequenceFile文件（Hadoop提供的 SequenceFile是一个由二进制序列化过的key/value的字节流组成的文本存储文件）
和其他符合Hadoop InputFormat格式的文件。
"""

'\n第一种：读取一个外部数据集。\n比如，从本地文件加载数据集，或者从HDFS文件系统、HBase、Cassandra、Amazon S3等外部数据源中加载数据集。\nSpark可以支持文本文件、SequenceFile文件（Hadoop提供的 SequenceFile是一个由二进制序列化过的key/value的字节流组成的文本存储文件）\n和其他符合Hadoop InputFormat格式的文件。\n'

In [2]:
"""hadoop hdfs 伪分布式
http://dblab.xmu.edu.cn/blog/install-hadoop/
Hadoop配置文件说明
Hadoop 的运行方式是由配置文件决定的（运行 Hadoop 时会读取配置文件），
因此如果需要从伪分布式模式切换回非分布式模式，需要删除 core-site.xml 中的配置项。

此外，伪分布式虽然只需要配置 fs.defaultFS 和 dfs.replication 就可以运行（官方教程如此），
不过若没有配置 hadoop.tmp.dir 参数，则默认使用的临时目录为 /tmp/hadoo-hadoop，
而这个目录在重启时有可能被系统清理掉，导致必须重新执行 format 才行。所以我们进行了设置，
同时也指定 dfs.namenode.name.dir 和 dfs.datanode.data.dir，否则在接下来的步骤中可能会出错。
"""
# 配置完成后，执行 NameNode 的格式化:
# $ ./bin/hdfs namenode -format

# 接着开启 NameNode 和 DataNode 守护进程。
# $ ./sbin/start-dfs.sh

"""
启动时可能会出现如下 WARN 提示：WARN util.NativeCodeLoader: 
Unable to load native-hadoop library for your platform… using builtin-java classes where applicable WARN 提示
可以忽略，并不会影响正常使用。
"""

# 启动完成后，可以通过命令 `jps` 来判断是否成功启动，

'\n启动时可能会出现如下 WARN 提示：WARN util.NativeCodeLoader: \nUnable to load native-hadoop library for your platform… using builtin-java classes where applicable WARN 提示\n可以忽略，并不会影响正常使用。\n'

In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
lines = sc.textFile("hdfs://localhost:9000/user/shilinlee/word.txt")
# >>> lines = sc.textFile("/user/shilinlee/word.txt")
# >>> lines = sc.textFile("word.txt")

In [5]:
lines.flatMap(lambda x: x.split()).map(lambda line: (line, 1)).reduceByKey(lambda a, b: a+b).collectAsMap()

{'Spark': 1, 'world': 1, 'good': 1, 'hello': 3, 'Scala': 1, 'morning': 1}

In [6]:
"""
第二种：调用SparkContext的parallelize方法，在Driver中一个已经存在的集合（数组）上创建。
"""

'\n第二种：调用SparkContext的parallelize方法，在Driver中一个已经存在的集合（数组）上创建。\n'

In [7]:
nums = [1,2,3,4,5]
rdd = sc.parallelize(nums)

In [8]:
# 会调用persist(MEMORY_ONLY)，但是，语句执行到这里，并不会缓存rdd，这时rdd还没有被计算生成, 
# 而是要等到遇到第一个行动操作触发真正计算以后，才会把计算结果进行持久化
rdd.cache()

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:194

In [9]:
rdd.count()

5

In [10]:
rdd.collect() # 从缓存中读取

[1, 2, 3, 4, 5]

In [11]:
rdd.unpersist()  # 主动清除缓存

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:194

In [12]:
rdd.collect() # 从头计算

[1, 2, 3, 4, 5]